In [1]:
print("Hello World")

Hello World


In [2]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

result = classifier("I love you")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}") 


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

c:\Users\RSass\ml\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\RSass\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

label: POSITIVE, with score: 0.9999


In [3]:
classifier('I hate you!')

[{'label': 'NEGATIVE', 'score': 0.9987472295761108}]

In [4]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\RSass\AppData\Roaming\nltk_data...


True

In [7]:
import PyPDF2
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

def preprocess_text(text):
    # Remove special characters and extra whitespace
    text = ' '.join(text.split())
    return text

def analyze_sentiment(text):
    # Using NLTK's VADER for sentiment analysis
    sia = SentimentIntensityAnalyzer()
    sentiment_scores = sia.polarity_scores(text)
    
    # Using Hugging Face Transformers for a more nuanced analysis
    classifier = pipeline("sentiment-analysis")
    result = classifier(text[:512])  # Most models have a max input length
    
    return sentiment_scores, result[0]

def interpret_results(vader_scores, hf_result):
    print(f"VADER Sentiment Scores: {vader_scores}")
    print(f"Hugging Face Sentiment: {hf_result}")
    
    # Additional interpretation
    compound_score = vader_scores['compound']
    if compound_score >= 0.05:
        print("VADER interpretation: Positive sentiment")
    elif compound_score <= -0.05:
        print("VADER interpretation: Negative sentiment")
    else:
        print("VADER interpretation: Neutral sentiment")
    
    print(f"Hugging Face interpretation: {hf_result['label']} with {hf_result['score']:.2%} confidence")

def main(pdf_path):
    print(f"Analyzing resume: {pdf_path}")
    text = extract_text_from_pdf(pdf_path)
    preprocessed_text = preprocess_text(text)
    vader_scores, hf_result = analyze_sentiment(preprocessed_text)
    interpret_results(vader_scores, hf_result)

if __name__ == "__main__":
    pdf_path = "C:/Users/RSass/ml/code/daniels-resume-2024.pdf"  # Replace with the actual path to your resume
    main(pdf_path)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Analyzing resume: C:/Users/RSass/ml/code/daniels-resume-2024.pdf
VADER Sentiment Scores: {'neg': 0.029, 'neu': 0.906, 'pos': 0.065, 'compound': 0.7506}
Hugging Face Sentiment: {'label': 'POSITIVE', 'score': 0.8009523749351501}
VADER interpretation: Positive sentiment
Hugging Face interpretation: POSITIVE with 80.10% confidence
